In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#   for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import time
import Levenshtein as lev

In [ ]:
train = pd.read_csv('/kaggle/input/shopee-product-matching/train.csv')
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/shopee-product-matching/test.csv')
test.head()

# Finding similar phashes

From the EDA notebook, we know that group 3627744656 has 51 matching items. Let's try using Levenshtein distance as a way of finding similar phashes in that group.

In [ ]:
same_group = train.iloc[np.where(train['label_group'] == 3627744656)]
same_group.head()

In [ ]:
distance = lev.distance('be9c95c3c16a631c', 'be9c90c3c16f631c')
print(distance)
ratio = lev.ratio('be9c95c3c16a631c', 'be9c90c3c16f631c')
print(ratio)

distance = lev.distance('be9c95c3c16a631c', 'be9d95c1c16a631c')
print(distance)
ratio = lev.ratio('be9c95c3c16a631c', 'be9d95c1c16a631c')
print(ratio)

In [ ]:
count = 0
for phash in same_group['image_phash']:
    if lev.distance('be9c95c3c16a631c', phash) <= 2:
        print(phash)
        count += 1
        
print(count)

# Putting this together with our submission

In [ ]:
def similar_phash(phash):
    matching = []
    for ind in test.index:
        #print(ind)
        #print(test['image_phash'][ind])
        if lev.distance(phash, test['image_phash'][ind]) <= 2:
            matching.append(ind)
            
    #print(matching)
    return matching

In [ ]:
start_time = time.time()

ids = []
item_group = [] # tracks which group an item is in
groups = {} # tracks which items are in each group
for ind in test.index:
    
    indices_phash = np.where(test['image_phash'] == test['image_phash'][ind])[0]
    indices_title = np.where(test['title'] == test['title'][ind])[0]
    indices_image = np.where(test['image'] == test['image'][ind])[0]
    indices_phash_similar = similar_phash(test['image_phash'][ind])
    indices = set(indices_phash).union(set(indices_title)).union(set(indices_image)).union(set(indices_phash_similar))
    
    # check if an item this matches already has a group
    match_ids = list(test.loc[indices]['posting_id'])
    if set(match_ids).intersection(ids):
        existing_group = item_group[ids.index(list(set(match_ids).intersection(ids))[0])]
        item_group.append(existing_group)
        groups[existing_group] = set(groups[existing_group]).union(match_ids)
    else:
        item_group.append(len(groups))
        groups[len(groups)] = match_ids
        
    ids.append(test['posting_id'][ind])
    
matches = [' '.join(list(groups[ind])) for ind in item_group]

print(time.time() - start_time)

In [ ]:
submission = pd.DataFrame({'posting_id': ids, 'matches': matches})
submission.to_csv('submission.csv', index = False)
submission

# Checking on Train Set

In [ ]:
#def similar_phash(phash):
#    matching = []
#    for ind in train.index:
#        if lev.distance(phash, train['image_phash'][ind]) <= 2:
#            matching.append(ind)
            
#    return matching

In [ ]:
#start_time = time.time()

#train_ids = []
#train_item_group = [] # tracks which group an item is in
#train_groups = {} # tracks which items are in each group
#for ind in train.index:
    
#    indices_phash = np.where(train['image_phash'] == train['image_phash'][ind])[0]
#    indices_title = np.where(train['title'] == train['title'][ind])[0]
#    indices_image = np.where(train['image'] == train['image'][ind])[0]
#    indices_phash_similar = similar_phash(train['image_phash'][ind])
#    indices = set(indices_phash).union(set(indices_title)).union(set(indices_image)).union(set(indices_phash_similar))
    
    # check if an item this matches already has a group
#    match_ids = list(train.loc[indices]['posting_id'])
#    if set(match_ids).intersection(train_ids):
#        existing_group = train_item_group[train_ids.index(list(set(match_ids).intersection(train_ids))[0])]
#        train_item_group.append(existing_group)
#        train_groups[existing_group] = set(train_groups[existing_group]).union(match_ids)
#    else:
#        train_item_group.append(len(train_groups))
#        train_groups[len(train_groups)] = match_ids
        
#    train_ids.append(train['posting_id'][ind])
    
#train_matches = [' '.join(list(train_groups[ind])) for ind in train_item_group]

#print(time.time() - start_time)

In [ ]:
#similar_phash('be9c95c3c16a631c')

In [ ]:
#for ind in similar_phash('be9c95c3c16a631c'):
#    print(train['label_group'][ind])

In [ ]:
#train_matches[32164]